In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Loading Data

In [2]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [3]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked


# One hot encoding

In [4]:
df = pd.get_dummies(df)
df.head(3)

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,28.0,0,0,79.53,31.1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,1,0
1,33.0,0,0,78.44,23.9,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,42.0,0,0,103.00,40.3,1,0,0,0,1,...,0,1,0,0,1,0,1,0,0,0


# Preprocessing v2

In [5]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [6]:
xgb_params = {'n_estimators': 476,
                 'max_depth': 4,
                 'learning_rate': 0.11543473253338146,
                 'min_child_weight': 2,
                 'gamma': 0.0018317182960446814,
                 'subsample': 0.5873657682934323,
                 'colsample_bytree': 0.9688354997685595,
                 'reg_alpha': 0.00229271071097547,
                 'reg_lambda': 0.004398118451936747}

In [7]:
xgb_model = xgb.XGBClassifier(**xgb_params)

In [8]:
xgb_model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.9688354997685595, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              gamma=0.0018317182960446814, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.11543473253338146, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=476, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0.00229271071097547,
              reg_lambda=0.004398118451936747, ...)

In [9]:
y_pred = xgb_model.predict_proba(test_new)[:, 1]
y_pred

array([2.5620449e-02, 6.1894953e-02, 1.1817585e-04, ..., 2.6924305e-05,
       7.7504676e-04, 5.8862151e-05], dtype=float32)

In [10]:
# since we dropped id column from our test dataframe earlier but we need it for submission file so we're reloading it
test_idx = pd.read_csv(BASE_DIR / "test.csv").id
test_idx

0        15304
1        15305
2        15306
3        15307
4        15308
         ...  
10199    25503
10200    25504
10201    25505
10202    25506
10203    25507
Name: id, Length: 10204, dtype: int64

In [11]:
submission = pd.DataFrame(data={"id": test_idx, "stroke": y_pred})
submission.head()

,id,stroke
0,15304,0.025620
1,15305,0.061895
2,15306,0.000118
3,15307,0.069369
4,15308,0.001693


In [12]:
submission.to_csv("submission.csv", index=False)